# Algo-trading: data preparation


> Author: <b>Nicola Bini</b>
>
> Date:   <b>06/17/2021</b>
>
>
> <b>Team 2:</b>
> <i>
> <br>Nicola Bini
> <br>Felipe Domingues
> <br>Tri Dung Dinh
> <br>Manuel Echazarra
> </i>

# Summary

Use signals generated previously by neural network to backtest the strategy on the testing data

In [1]:
# Import libraries
import bt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
# Import ML signals

tickets    = ["OCGN", "HGEN", "VUZI", "DNN"]
start_date = "2018-06-25"
end_date   = "2021-06-08"
test_data  = bt.get(tickets, start=start_date, end=end_date)

test_signal_OCGN = pd.read_csv("test_signal_OCGN.csv")
test_signal_HGEN = pd.read_csv("test_signal_HGEN.csv")
test_signal_VUZI = pd.read_csv("test_signal_VUZI.csv")
test_signal_DNN  = pd.read_csv("test_signal_DNN.csv")

test_signal_OCGN['date'] = test_signal_OCGN['date'] .astype(np.datetime64)
test_signal_HGEN['date'] = test_signal_HGEN['date'] .astype(np.datetime64)
test_signal_VUZI['date'] = test_signal_VUZI['date'] .astype(np.datetime64)
test_signal_DNN['date']  = test_signal_DNN['date']  .astype(np.datetime64)

test_signal_OCGN = test_signal_OCGN.set_index("date")
test_signal_HGEN = test_signal_HGEN.set_index("date")
test_signal_VUZI = test_signal_VUZI.set_index("date")
test_signal_DNN  = test_signal_DNN.set_index("date")


In [3]:
signals = pd.concat([test_signal_OCGN, test_signal_HGEN, test_signal_VUZI, test_signal_DNN], axis=1)

In [4]:
# Check data structure
test_data

,ocgn,hgen,vuzi,dnn
Date,,,,
2018-06-25,162.000000,3.150000,7.300000,0.51
2018-06-26,153.600006,3.150000,7.450000,0.50
2018-06-27,147.000000,3.150000,7.080000,0.49
2018-06-28,149.399994,3.250000,7.250000,0.48
2018-06-29,150.000000,3.350000,7.450000,0.50
...,...,...,...,...
2021-06-02,9.455000,19.360001,19.990000,1.26
2021-06-03,8.890000,18.240000,18.870001,1.28
2021-06-04,8.720000,18.580000,18.730000,1.37


In [5]:
# Check signals
signals

,OCGN,HGEN,VUZI,DNN
date,,,,
2018-06-25,0,0,1,0
2018-06-26,0,1,1,0
2018-06-27,0,0,1,0
2018-06-28,0,0,1,0
2018-06-29,0,0,0,0
...,...,...,...,...
2021-06-02,1,0,1,0
2021-06-03,1,1,0,0
2021-06-04,0,0,0,0


In [6]:
# Set weights
for col in signals:
    for i, row in signals.iterrows():
        if signals.loc[i, col] == 1:
            signals.loc[i, col] = -0.25
        elif signals.loc[i, col] == 0:
            signals.loc[i, col] = 0.25
        else:
            signals.loc[i, col] = "ERROR!!"
            
for col in signals:
    signals[col.lower()] = signals[col]
    signals.drop(col,axis=1, inplace=True)

In [7]:
# Check for errors
signals.value_counts()

ocgn   hgen   vuzi   dnn  
 0.25  -0.25   0.25   0.25    115
        0.25   0.25   0.25     96
       -0.25  -0.25   0.25     91
        0.25  -0.25   0.25     89
       -0.25   0.25  -0.25     59
        0.25   0.25  -0.25     52
       -0.25  -0.25  -0.25     41
-0.25  -0.25   0.25   0.25     40
 0.25   0.25  -0.25  -0.25     39
-0.25   0.25   0.25   0.25     32
              -0.25   0.25     31
       -0.25  -0.25   0.25     22
               0.25  -0.25     12
        0.25  -0.25  -0.25      9
               0.25  -0.25      9
       -0.25  -0.25  -0.25      7
dtype: int64

In [8]:
#bt.algos.SelectWhere(),

In [9]:
# Create strategies
ml_strategy = bt.Strategy('ml_strategy', [bt.algos.WeighEqually(signals),
                                          bt.algos.Rebalance()])

ml_strategy = bt.Strategy('ml_strategy', [
                                          #bt.algos.SelectWhere(signals),
                                          bt.algos.WeighTarget(signals),
                                          bt.algos.Rebalance()])

# benchmark   = bt.Strategy("benchmark_buy_hold",[bt.algos.RunWeekly(),
#                                                 bt.algos.SelectAll(),
#                                                 bt.algos.WeighEqually(),
#                                                 bt.algos.Rebalance()])

TypeError: __init__() takes 1 positional argument but 2 were given

In [ ]:
ml_backtest = bt.Backtest(ml_strategy, test_data)
benchmark_backtest = bt.Backtest(benchmark, test_data)
ml_res      = bt.run(benchmark_backtest, ml_backtest)

In [ ]:
ml_res.plot(figsize=(15, 10),color='orange')
plt.show()

In [ ]:
ml_res.display()
ml_res.plot_security_weights()